In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder\
    .master("local")\
    .appName("lesson_15")\
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 15:49:24 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/05 15:49:24 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/05 15:49:24 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/05 15:49:24 WARN util.Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [11]:
df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("/datalake/bronze/dshop/clients/2022-02-24/clients.csv")

In [12]:
df.first()
df.columns

['client_id', 'fullname', 'location_area_id']

In [13]:
df.repartition(6).explain()   # distribute data between 6 partitions round robin   (Num partitions >= Num worker numbers)

== Physical Plan ==
Exchange RoundRobinPartitioning(6), REPARTITION_WITH_NUM, [id=#107]
+- FileScan csv [client_id#64,fullname#65,location_area_id#66] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/datalake/bronze/dshop/clients/2022-02-24/clients.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<client_id:int,fullname:string,location_area_id:int>




In [14]:
df.repartition(6, "location_area_id").explain()   # distribute data between 6 partition by location_area_id

== Physical Plan ==
Exchange hashpartitioning(location_area_id#66, 6), REPARTITION_WITH_NUM, [id=#114]
+- FileScan csv [client_id#64,fullname#65,location_area_id#66] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/datalake/bronze/dshop/clients/2022-02-24/clients.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<client_id:int,fullname:string,location_area_id:int>




In [16]:
df.coalesce(2).explain()     # unite partitions (merge) - as result 2 partitions

== Physical Plan ==
Coalesce 2
+- FileScan csv [client_id#64,fullname#65,location_area_id#66] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/datalake/bronze/dshop/clients/2022-02-24/clients.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<client_id:int,fullname:string,location_area_id:int>


